# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import re
import pickle
import time

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'stopwords', 'wordnet', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mamadcamzis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterDatabase.db')
df = pd.read_sql_table("DisasterTable", con=engine)


In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message
y = df.iloc[:, 4:]

In [5]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [6]:
y.columns.tolist()

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [7]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """Tokenize text data
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    word_text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(word_text)
    word_tokens = [word for word in tokens if word not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in word_tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([("vect", CountVectorizer(tokenizer=tokenize)), 
                     ("tfidf", TfidfTransformer()),
                     ("mclf", MultiOutputClassifier(RandomForestClassifier()))])

In [ ]:
X.shape[0]*.25

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [ ]:
start = time.time()
pipeline.fit(X_train, y_train)
print(time.time()-start)

In [ ]:
pipeline.get_params().keys()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred

In [ ]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))
   
    

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params().keys()

In [ ]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'mclf__estimator__n_estimators': [50, 100],
      
        }


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


In [ ]:
start = time.time()
cv.fit(X_train, y_train)
print(time.time()-start)

In [ ]:
pipeline.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) > 0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
        return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

  
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [ ]:
pipeline.get_params().keys()

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'mclf__estimator__n_estimators': [50, 100],
        'features__transformer_weights': (
            {'features__text_pipeline': 1, 'features__starting_verb': 0.5},
            {'features__text_pipeline': 0.5, 'features__starting_verb': 1},
            {'features__text_pipeline': 0.8, 'features__starting_verb': 1},
        )
      
        }


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


In [ ]:
#list(pipeline.get_params().keys())

In [ ]:
#start = time.time()
#cv.fit(X_train, y_train)
#print(time.time()-start)

This pipeline takes amoung three days. Clearly I do not 
take it for deployment so that I am using an AdaBooos

In [12]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

  
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('aclf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [13]:
parameters = {
    'aclf__estimator__learning_rate': [0.1, 0.2, 0.3],
    'aclf__estimator__n_estimators': [100, 200]
}

In [14]:
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, n_jobs=-1, verbose=3)

In [16]:
start = time.time()
cv.fit(X_train, y_train)
print(time.time()-start)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
for col in range(y_pred.shape[1]):
    pred_columns = y_pred[:, col]
    true_columns = y_test.iloc[:, col]
   
    print(classification_report(true_columns, pred_columns))

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('disaster_notebook_adaboost_model.sav', 'wb'))

In [ ]:
model = pickle.load(open('disaster_notebook_adaboost_model.sav', 'rb'))